In [1]:
## Script to analyze Turbo Typing data

Notes on where I'm leaving off:   
- 9/22/24 Just wrote code that removes the extra spaces before characters from the imported data without deleting actual spaces. Next I want to use the collapsed sentences with no extra s
- spaces to use the diff function and compare them.  
- 9/23/2024 Basically same goal for next time. I had to make a new column that replaces the multi-character key names with special characters for the keyData from the participants, which was completed today.
- 9/24/2024 Isolated correct keypress times. Next time, calculate ttk for correct presses, mean ttk, and the rest of the assessment variables. Also ask Patrick the best way to organize and write code.

In [2]:
import os
import numpy as np
import pandas as pd
import math
import glob
import editdistance
from scipy import stats
import diff_match_patch as dmp_module
dmp = dmp_module.diff_match_patch()
import copy
from scipy.stats import pearsonr
from scipy.stats import variation
import typingmod as typ

In [3]:
# The imported data from Turbo Typing stores listed data as a long string. 
# This splits it up into actual lists.
def str_to_list(dataframe, columns):
    for col in columns:
        data_str = dataframe[col]
        if type(data_str.iloc[0]) == list:
            pass
        elif type(data_str.iloc[0]) == str:
            if data_str.iloc[0][0].isdigit() == True:
                data_list = data_str.apply(lambda trial: 
                                           [int(char) for char in trial.split(',')])
                dataframe[col] = data_list
            elif data_str.iloc[0][0].isdigit() == False:
                data_list = [x.split(',') for x in data_str]
                dataframe[col] = data_list
        else:
            print('Column must have data in string form.')

# Removes extra spaces before characters in keypress data.
def no_extra_spaces(dataframe, columns):
    for col in columns:
        no_space = dataframe[col].apply(lambda trial: 
                                        [char.replace(" ","") if char.isspace() 
                                         != True else ' ' for char in trial])
        dataframe[col] = no_space

# Changes multi-chracter keynames (ie. LeftShift and Backspace) to special
# characters.
def multi_to_special(dataframe, column):
    data = dataframe[column]
    edit_data = []
    for trial in data:
        edit_trial = copy.deepcopy(trial) # Pandas can't deep copy lists in dataframes, so this is a work around.
        for index, key in enumerate(edit_trial):
            if len(key) == 1:
                pass
            if len(key) > 1:
                if key in ['LeftShift', 'RightShift']:
                    edit_trial[index] = '#'
                elif key == 'Backspace':
                    edit_trial[index] = '-'
                elif key == 'Return':
                    edit_trial[index] = '^'
                else:
                    edit_trial[index] = '~' # '~' replaces any other keys longer than one character.
        edit_data.append(edit_trial)
    return edit_data

# Calculates time to keypress (TTK) for all presses in one trial.
# TTK is the reaction time for the first press and the interkey 
# interval for any subsequent presses.
def ttk(row, column):
    presses = pd.DataFrame(row.loc[column])
    ttk_df = presses.sub(presses.shift(fill_value=0))
    ttk_list = ttk_df.values.flatten().tolist()
    return ttk_list

# Creates a list of keys that would have been pressed if the sentence was typed
# with no errors (ie. no backspacing). Uses single, special characters to 
# represent multi-character key names (ie. '^' for 'Return').
def sentence_to_keydata(string):
    curr_keyData = []
    for index, data in enumerate(string):
        if data.isupper() == True:
            curr_keyData.append('#') # '#' Replaces 'Right/LeftShift'
            curr_keyData.append(data)
        elif data.isupper() == False and data.isalpha() or data.isspace() == True:
            curr_keyData.append(data)
        elif data == '?':
            curr_keyData.append('#')
            curr_keyData.append(data)
            curr_keyData.append('^')
        elif data == '.':
            curr_keyData.append(data)
            curr_keyData.append('^') # '^' Replaces 'Return'
    return curr_keyData

# # Splits up keyDataConverted data from a string to a list of pressed keys.
# def split_str(row):
#     keys_str = row.loc['keyDataConverted']
#     keys_list = keys_str.split(',')
#     return keys_list

In [4]:
# Setting save directory
save_dir = r'/Users/rubi/Desktop/Github/typingexp/typing_task_analysis/'\
           'figures'
save_dir

'/Users/rubi/Desktop/Github/typingexp/typing_task_analysis/figures'

In [5]:
# Getting subject data folders from server.
server = r'/Volumes/greenhouse/typingtask_data/subject_data'
os.chdir(server)
folders = os.listdir()

# Looping through subject folders, getting appropriate paths to data, 
# and making sID list.
sub_folders = list(filter(lambda x: x.startswith('s', 0, 1), folders))
all_turbo = pd.DataFrame()
ID_list = []
for sub in sub_folders:
    sub_folder = r'/Volumes/greenhouse/typingtask_data/'\
                 'subject_data/%s/turbotyping_data' % sub
    os.chdir(sub_folder)
    sID = sub.split('_', 1)[0]
    turbo = pd.read_csv(glob.glob('*_datafile.tsv')[0], sep='\t')
    all_turbo = pd.concat([all_turbo, turbo])
    ID_list.append(sID)

all_turbo = all_turbo.reset_index()

In [6]:
ID_list

['s262',
 's261',
 's240',
 's217',
 's176',
 's267',
 's263',
 's254',
 's209',
 's276',
 's278',
 's279',
 's282',
 's283',
 's286',
 's302',
 's304',
 's306',
 's305',
 's309',
 's311',
 's009',
 's020',
 's207',
 's313']

In [7]:
# Transforms listed data stored in long strings to actual lists.
str_to_list(all_turbo, ['timeData', 
                        'keyData', 
                        'keyDataConverted', 
                        'timeDataUp',
                        'keyDataUp'])

# Removes extra spaces before characters in keypress data columns.
no_extra_spaces(all_turbo, ['keyData', 'keyDataConverted', 'keyDataUp'])

# Replaces multi-character key names with single, special characters from key
# press data typed by participants. 
no_multichar_keys = multi_to_special(all_turbo, 'keyDataConverted')
all_turbo.insert(11, 'keyDataSpecialChar', no_multichar_keys)

# Creates column that represents the correct key press sequence if there were
# no errors (uses special characters for multi-character key names).
current_keyData = [sentence_to_keydata(x) for x in all_turbo['currentSentence']]
all_turbo.insert(8, 'keyDataCurrent', current_keyData)

In [8]:
# Compares the correct sequence of keypresses with what was typed during the 
# trial and returns the difference between the two.
string_diff = all_turbo.apply(lambda row: 
                            dmp.diff_main(''.join(row.loc['keyDataSpecialChar']), 
                                          ''.join(row.loc['keyDataCurrent'])),
                            axis=1)

# Separates the returned differences into individual characters.
diffs_bychar = []
for trial in string_diff:
    diff_bychar_trial = []
    for diff in trial:
        if len(diff[1]) == 1:
            diff_bychar_trial.append([diff[0], diff[1]])
        elif len(diff[1]) > 1:
            for char in diff[1]:
                diff_bychar_trial.append([diff[0], char])
    diffs_bychar.append(diff_bychar_trial)

# Differences for errors where participants swap two letters are solved as: 
# letter 2 subtraction, letter 1 retention, letter 2 addition. Because  
# both letters 1 and 2 need to be identified as incorrect keypresses, this code 
# changes the difference to show: letter 2 subtraction, letter 1 subtraction.
diffs_bychar_fixswap = []
for trial in diffs_bychar:
    chardiffs_trial = []
    char_toskip = 0
    for index, char in enumerate(trial):
        if char_toskip != 0:
            char_toskip = char_toskip - 1
            continue
        if char[0] == 0:
            if index == (len(trial) - 1) or index == 0:
                chardiffs_trial.append(char)
            else:
                if trial[index - 1][0] == -1 and trial[index + 1][0] == 1:
                    if trial[index - 1][1] == trial[index + 1][1]:
                        chardiffs_trial.append([-1, char[1]])
                        char_toskip = 1
                else:
                    chardiffs_trial.append(char)
        else:
            chardiffs_trial.append(char)
    diffs_bychar_fixswap.append(chardiffs_trial)

In [9]:
# Isolates characters and indices in original keyData/timeData lists for correct 
# keypresses and adds both to new columns.
corr_keys = []
corr_indices = []
for trial in diffs_bychar_fixswap:
    corr_keys_trial = []
    corr_indices_trial = []
    for index, char in enumerate(trial):
        if char[0] == 0:
            corr_keys_trial.append(char[1])
            corr_indices_trial.append(index)
        else:
            pass
    corr_keys.append(corr_keys_trial)
    corr_indices.append(corr_indices_trial)
    
all_turbo.insert(12, 'keyDataCorrect', corr_keys)
all_turbo.insert(13, 'keyIndicesCorrect', corr_indices)

In [10]:
# Isolates keypress times for correct keypresses and adds to a new column.
def corr_time(trial):
    times = trial.loc['timeData']
    indices = trial.loc['keyIndicesCorrect']
    matched_indices =[]
    for index, time in enumerate(times):
        if index in indices:
            matched_indices.append(time)
    return matched_indices

corr_times = all_turbo.apply(corr_time, axis=1)
all_turbo.insert(10, 'timeDataCorrect', corr_times)

In [11]:
# Calculates edit distance of each trial and stores to an additional column.
edit_dist = all_turbo.apply(lambda row: 
                            editdistance.eval(row.loc['currentSentence'], 
                                              row.loc['typedSentence']), 
                            axis=1)
all_turbo['editDistance'] = edit_dist

# Calculates TTK for every character typed in each trial and stores to 
# an additional column.
ttks = all_turbo.apply(lambda row: ttk(row, 'timeData'), axis=1)
all_turbo['ttk'] = ttks

# Calculates TTK for every character correctly typed in each trial and stores to 
# an additional column.
ttks_corr = all_turbo.apply(lambda row: ttk(row, 'timeDataCorrect'), axis=1)
all_turbo['ttkCorrect'] = ttks_corr

# Calculates mean TTK for each trial and stores to an additional column.
mean_ttk = [np.mean(x) for x in all_turbo['ttk']]
all_turbo['ttkMean'] = mean_ttk

# Calculates mean correct TTK for each trial and stores to an additional column.
mean_ttk_corr = [np.mean(x) for x in all_turbo['ttkCorrect']]
all_turbo['ttkMeanCorrect'] = mean_ttk_corr

# Calculates sum of all TTKs for each trial and stores to an additional column.
sum_ttk = [np.sum(x) for x in all_turbo['ttk']]
all_turbo['ttkSum'] = sum_ttk
all_turbo['ttkMean']

# Calculates sum of all correct TTKs for each trial and stores to an additional 
# column.
sum_ttk_corr = [np.sum(x) for x in all_turbo['ttkCorrect']]
all_turbo['ttkSumCorrect'] = sum_ttk_corr

In [12]:
# Calculates IES
def ies(row):
    mean_corr_rt = row['ttkMeanCorrect']
    pc = len(row['ttkCorrect'])/len(row['ttk'])
    return mean_corr_rt/pc

ies = all_turbo.apply(lambda row: ies(row), 
                      axis=1)
all_turbo['ies'] = ies

In [13]:
# Calculates rate correct score (RCS) for each trial and stores in column.
rcs = all_turbo.apply(lambda row: 
                      len(row.loc['ttkCorrect'])/row.loc['ttkSum'],
                      axis=1)
all_turbo['rcs'] = rcs

In [14]:
# Calculates linear integrated speed-ccuracy score (LISAS) and stores in column.
def lisas(row):
    rt_mean = row.loc['ttkMeanCorrect']
    rt_std = np.std(row.loc['ttkCorrect'])
    pe = 1-(len(row['ttkCorrect'])/len(row['ttk']))
    pe_std = np.sqrt(pe*(1-pe))
    return rt_mean + (np.std(rt_mean)/pe_std)*pe

lisas_output = all_turbo.apply(lambda row: lisas(row), axis=1)
all_turbo['lisas'] = lisas_output

/var/folders/42/sqkdrmx52wgfnxt8ycz1hhlc0000gn/T/ipykernel_61601/268902196.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return rt_mean + (np.std(rt_mean)/pe_std)*pe


In [15]:
# # Calculates BIS
# # by_sub = all_turbo.groupby('participantID')
# z_score = stats.zscore
# # by_sub = pd.DataFrame(all_turbo.groupby('participantID').filter(lambda x: print(x['trialNumber'])))
# ttkmean_bysub = pd.DataFrame(all_turbo[all_turbo['trialNumber'] >= 0].groupby('participantID').ttkMean)
# # totalttkmean_bysub = ttkmean_bysub
# # by_sub = by_sub.reset_index().rename(columns = {'ttk':'all_ttk'})
# # ttkmean_bysub
# # test = ttkmean_bysub.
# # test = ttkmean_bysub[1][0].mean()
# # test
# test = ttkmean_bysub[1].apply(lambda x: stats.zscore(x))
# # stats.zscore(ttkmean_bysub[1][0])
# # ttkmean_bysub
# # by_sub[1][0]
# # test
# # by_sub['all_ttk'].apply(stats.zscore)
# test = test.transpose()
# test

In [16]:
# Isolates keypress times for correctly typed alphabet characters (a-z) and adds 
# to a new column.
def alpha_time(row):
    trial = row.loc['keyDataCorrect']
    times = row.loc['timeData']
    alpha_indices = [index for (index, char) in enumerate(trial) if char.isalpha() == True]
    alpha_time = [time for (index, time) in enumerate(times) if index in alpha_indices]
    return alpha_time

alpha_times = all_turbo.apply(alpha_time, axis=1)
all_turbo.insert(11, 'timeDataAlpha', alpha_times)

# Calculates IKI (excludes first TTK) for correctly typed alphabet characters 
# (a-z) in each trial and stores to a new column.
ttks_alpha = all_turbo.apply(lambda row: ttk(row, 'timeDataAlpha'), axis=1)
ikis_alpha = [ttk[1:] for ttk in ttks_alpha]
all_turbo['ikiAlpha'] = ikis_alpha

# Calculates mean IKI for each trial and stores to new column. 
all_turbo['meanikiAlpha'] = [np.mean(ikis) for ikis in all_turbo.ikiAlpha]
all_turbo['sdikiAlpha'] = [np.std(ikis) for ikis in all_turbo.ikiAlpha]
all_turbo['cvikiAlpha'] = [variation(ikis) for ikis in all_turbo.ikiAlpha]

/Users/rubi/miniconda3/envs/jlab/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/rubi/miniconda3/envs/jlab/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/rubi/miniconda3/envs/jlab/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/rubi/miniconda3/envs/jlab/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/rubi/miniconda3/envs/jlab/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [17]:
# Calculates individual Mean IKI for correctly typed alphabet characters.
meanikis_alpha = all_turbo.groupby('participantID').meanikiAlpha.agg('mean').reset_index()

# Divides values by 1000 to put on same scale as mean IKIs from in-house typing
# task. 
def divby1000(value):
    return value/1000
meanikis_alpha['meanikiAlpha'] = meanikis_alpha.meanikiAlpha.transform(divby1000)
# meanikis_alpha

In [18]:
# Creating dataframe of subject score averages for each IES, RCS, and LISAS.
subject_scores = pd.DataFrame()
subject_scores['sID'] = ID_list
subject_scores = subject_scores.sort_values(by=['sID']).reset_index(drop = True)

# Adding mean IES scores.
mean_ies = all_turbo.groupby('participantID').ies.agg(['mean'])
mean_ies = mean_ies.sort_values(by=['participantID'])
subject_scores['mean_ies'] = mean_ies.reset_index()['mean']

# Adding mean IKIs for alphabetic characters.
meanikis_alpha = all_turbo.groupby('participantID').meanikiAlpha.agg('mean').reset_index()
def divby1000(value): # Divides values by 1000 to put on same scale as mean IKIs  
    return value/1000 # from in-house typing task.
meanikis_alpha['meanikiAlpha'] = meanikis_alpha.meanikiAlpha.transform(divby1000)
meanikis_alpha = meanikis_alpha.sort_values(by=['participantID'])
subject_scores['meaniki_alpha'] = meanikis_alpha.reset_index()['meanikiAlpha']

# Adding SD IKIs for alphabetic characters.
sdikis_alpha = all_turbo.groupby('participantID').sdikiAlpha.agg('mean').reset_index()
def divby1000(value): # Divides values by 1000 to put on same scale as mean IKIs  
    return value/1000 # from in-house typing task.
sdikis_alpha['sdikiAlpha'] = sdikis_alpha.sdikiAlpha.transform(divby1000)
sdikis_alpha = sdikis_alpha.sort_values(by=['participantID'])
subject_scores['sdiki_alpha'] = sdikis_alpha.reset_index()['sdikiAlpha']

# Adding CV IKIs for alphabetic characters.
cvikis_alpha = all_turbo.groupby('participantID').cvikiAlpha.agg('mean').reset_index()
cvikis_alpha = cvikis_alpha.sort_values(by=['participantID'])
subject_scores['cviki_alpha'] = cvikis_alpha.reset_index()['cvikiAlpha']

# Importing individual global scores calculated from typing task analysis script.
globals_dir = save_dir + '/indiv_globalmetrics.csv'
indiv_globals = pd.read_csv(globals_dir).drop(['Unnamed: 0'], axis = 1)
indiv_globals = indiv_globals[indiv_globals['sID'].isin(ID_list)]
indiv_globals = indiv_globals.sort_values(by=['sID']).reset_index(drop=True)

subject_scores = pd.concat([subject_scores, indiv_globals], axis = 1)

In [19]:
### ADDRESS/EDIT THIS LATER ###

# Dropping data from new subjects that won't be included in first paper.
subject_scores = subject_scores.dropna()
subject_scores

,sID,mean_ies,meaniki_alpha,sdiki_alpha,cviki_alpha,sID,mean_IKI,sd_IKI,cv_IKI,mean_RT,sd_RT,cv_RT,mean_deltaiki,total_deltaiki,mean_deltart,total_error
0,s009,258.931703,0.155438,0.109777,0.700639,s009,0.128731,0.066846,0.518988,0.632401,0.065977,0.104103,-0.002352,-0.024184,-0.012846,18.0
1,s020,473.267482,0.302347,0.250453,0.826906,s020,0.177566,0.106714,0.600633,1.044394,0.236281,0.225705,0.000885,0.006824,-0.024261,45.0
2,s176,1682.851715,0.316734,0.241707,0.744973,s176,0.203994,0.085076,0.416814,0.910563,0.169042,0.185223,-0.004743,-0.040195,-0.021532,50.0
3,s207,360.885650,0.208614,0.141034,0.647481,s207,0.164631,0.085433,0.518637,0.869910,0.191214,0.219297,-0.000987,-0.006829,0.000042,55.0
4,s209,347.896285,0.216724,0.172572,0.786554,s209,0.134536,0.071296,0.529646,0.847502,0.132069,0.155484,-0.000357,-0.007503,-0.018694,43.0
5,s217,230.162116,0.157552,0.080985,0.503416,s217,0.136209,0.049105,0.360322,0.698351,0.123780,0.176868,0.000219,0.004319,-0.007439,18.0
6,s240,254.896620,0.151232,0.080480,0.528411,s240,0.153041,0.068443,0.446983,0.604788,0.129137,0.213071,-0.000728,-0.007394,-0.004336,10.0
7,s254,438.068837,0.301409,0.237287,0.781249,s254,0.264398,0.192641,0.728154,1.096890,0.234291,0.213069,0.001963,-0.018492,-0.004777,91.0
8,s261,346.531622,0.195963,0.142673,0.715762,s261,0.137397,0.065837,0.478910,0.731690,0.105624,0.144040,-0.002988,-0.031318,-0.010033,32.0
9,s262,227.275971,0.140412,0.084362,0.583585,s262,0.128899,0.069887,0.541886,0.654782,0.127911,0.194923,-0.001937,-0.007667,-0.019326,23.0


In [24]:
# Pearson correlations between mean IES and typing task metrics.
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_IKI']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['sd_IKI']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['cv_IKI']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_RT']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['sd_RT']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['cv_RT'])) 
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_deltaiki']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['total_deltaiki']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_deltart']))
print(pearsonr(subject_scores['mean_ies'],
               subject_scores['total_error']))

PearsonRResult(statistic=0.33564142727247237, pvalue=0.10883869954088668)
PearsonRResult(statistic=0.16235239853101952, pvalue=0.44847986892395253)
PearsonRResult(statistic=-0.016666184002187785, pvalue=0.9383898708634373)
PearsonRResult(statistic=0.3649907711058864, pvalue=0.07948042995426248)
PearsonRResult(statistic=0.19904111212656178, pvalue=0.3511192808934843)
PearsonRResult(statistic=0.008313540163602202, pvalue=0.9692458389120489)
PearsonRResult(statistic=-0.2219377649297897, pvalue=0.2972741383300595)
PearsonRResult(statistic=-0.24467171376619748, pvalue=0.24920883239328573)
PearsonRResult(statistic=-0.12192561729162074, pvalue=0.5703413399016535)
PearsonRResult(statistic=0.3160624723863282, pvalue=0.13242648362994738)


In [21]:
# Pearson correlation between mean, SD, and CV IKI from Turbo and Typing tasks.
print('Mean IKI:', pearsonr(subject_scores['meaniki_alpha'],
               subject_scores['mean_IKI']))
print('SD IKI:', pearsonr(subject_scores['sdiki_alpha'],
               subject_scores['sd_IKI']))
print('CV IKI:', pearsonr(subject_scores['cviki_alpha'],
               subject_scores['cv_IKI']))

# Making list of Pearson results. 
alpha_pearsons = [
    (pearsonr(subject_scores['meaniki_alpha'], 
              subject_scores['mean_IKI'])),
    (pearsonr(subject_scores['sdiki_alpha'],
              subject_scores['sd_IKI'])),
    (pearsonr(subject_scores['cviki_alpha'],
              subject_scores['cv_IKI']))]

# List of only p-values
alpha_pvals = [val[1] for val in alpha_pearsons]

# Running Bonferonni correction
bon_alphaps = [p*len(alpha_pvals) for p in alpha_pvals]
print('Bonferonni p-vals', bon_alphaps)

# Running False Discovery Rate test
print('FDR p-vals', stats.false_discovery_control(alpha_pvals))

Mean IKI: PearsonRResult(statistic=0.8346352275594588, pvalue=3.9686240840398434e-07)
SD IKI: PearsonRResult(statistic=0.7115157517971853, pvalue=9.680494713354588e-05)
CV IKI: PearsonRResult(statistic=0.6733589598700805, pvalue=0.00031050653486556156)
Bonferonni p-vals [1.190587225211953e-06, 0.00029041484140063765, 0.0009315196045966847]
FDR p-vals [1.19058723e-06 1.45207421e-04 3.10506535e-04]


In [22]:
## Pearson correlation between ... 
ies_pearsons = [
    (pearsonr(subject_scores['mean_ies'],
             subject_scores['mean_IKI'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['sd_IKI'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['cv_IKI'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_RT'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['sd_RT'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['cv_RT'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_deltaiki'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['total_deltaiki'])),
    (pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_deltart']))
] 

# List of only p-values
pvals = [val[1] for val in ies_pearsons]

# Running Bonferonni correction
bon_ps = [p*len(pvals) for p in pvals]

# Running False Discovery Rate test
stats.false_discovery_control(pvals)

array([0.48977415, 0.6727198 , 0.96924584, 0.48977415, 0.63201471,
       0.96924584, 0.63201471, 0.63201471, 0.73329601])

In [23]:
x = pearsonr(subject_scores['mean_ies'],
               subject_scores['mean_deltart'])